<a href="https://colab.research.google.com/github/shyakx/Air-Quality-Forecasting/blob/main/LSTM_%2B_Dense_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Air Quality Forecasting with Simplified LSTM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Load data
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')

# Drop NaNs
train_df.dropna(subset=['pm2.5'], inplace=True)

# Feature engineering: Extract datetime parts
for df in [train_df, test_df]:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month

# Use available columns (no 'cbwd' in current dataset)
features = ['TEMP', 'PRES', 'DEWP', 'Iws', 'hour', 'dayofweek', 'month']
target = 'pm2.5'

# Scaling
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

scaled_features = feature_scaler.fit_transform(train_df[features])
scaled_target = target_scaler.fit_transform(train_df[[target]])

scaled_df = pd.DataFrame(scaled_features, columns=features)
scaled_df['pm2.5'] = scaled_target

# Create sequences for time series forecasting
def create_sequences(data, target_column, window_size=48, forecast_horizon=6):
    X, y = [], []
    for i in range(len(data) - window_size - forecast_horizon):
        X.append(data.iloc[i:i+window_size][features].values)
        future_targets = data.iloc[i+window_size:i+window_size+forecast_horizon][target_column]
        y.append(future_targets.mean())
    return np.array(X), np.array(y)

# Create input-output pairs
window_size = 48
forecast_horizon = 6
X, y = create_sequences(scaled_df, 'pm2.5', window_size, forecast_horizon)

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)

# Define simple LSTM model
model = Sequential([
    LSTM(32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(16),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

# Train
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
val_preds = model.predict(X_val)
val_preds_original = target_scaler.inverse_transform(val_preds)
y_val_original = target_scaler.inverse_transform(y_val.reshape(-1, 1))
rmse = sqrt(mean_squared_error(y_val_original, val_preds_original))
print(f"Validation RMSE: {rmse:.2f}")

# Prepare test data
test_scaled = feature_scaler.transform(test_df[features])
X_test = [test_scaled[i:i+window_size] for i in range(len(test_scaled) - window_size - forecast_horizon)]
X_test = np.array(X_test)

# Predict on test set
test_preds_scaled = model.predict(X_test)
test_preds = target_scaler.inverse_transform(test_preds_scaled)

# Prepare final submission
submission = sample_submission.copy()
submission['pm2.5'] = np.concatenate([
    test_preds.flatten(),
    np.full(len(sample_submission) - len(test_preds), test_preds.flatten()[-1])
])
submission.to_csv('/content/submission_G.csv', index=False)
print("✅ Submission saved as submission_G.csv")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - loss: 0.0089 - val_loss: 0.0040 - learning_rate: 0.0010
Epoch 2/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 17s 41ms/step - loss: 0.0060 - val_loss: 0.0039 - learning_rate: 0.0010
Epoch 3/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 17s 41ms/step - loss: 0.0053 - val_loss: 0.0037 - learning_rate: 0.0010
Epoch 4/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 16s 40ms/step - loss: 0.0052 - val_loss: 0.0035 - learning_rate: 0.0010
Epoch 5/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0047 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 6/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0046 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 7/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - loss: 0.0044 - val_loss: 0.0033 - learning_rate: 0.0010
Epoch 8/100
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0044
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
404/404 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0044 - val_